In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 9.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 11.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 60.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 7.0 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-vkd0b386/overlay/local/bin
    changing mode of /tmp/pip-build-env-vkd0b386/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-vkd0b386/overlay/l

In [2]:
import re
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [3]:

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [4]:

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [5]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=1024, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [6]:
from google.colab import files
pd= files.upload()

Saving combined_data_amazon (1).csv to combined_data_amazon (1).csv


In [7]:
import pandas as pd
import random
df = pd.read_csv('combined_data_amazon (1).csv')


In [8]:
df = df[['reviews']]
df.head()

,reviews
0,"I always keep this product in my home, whether..."
1,Absolutely love this stuff! It’s a holistic wa...
2,Have been using this product for years. Has ma...
3,I like to take this every morning and there’s ...
4,I love this stuff. I use it for so many things...


In [9]:
#Step 6: Generating the Response

In [10]:
categories_list = [
        r"Blood and lymphatic system disorders",
        r"Cardiac disorders",
        r"Congenital, familial and genetic disorders",
        r"Ear and labyrinth disorders",
        r"Endocrine disorders",
        r"Eye disorders",
        r"Gastrointestinal disorders",
        r"General disorders and administration site conditions",
        r"Hepatobiliary disorders",
        r"Immune system disorders",
        r"Infections and infestations",
        r"Injury, poisoning, and procedural complications",
        r"Investigations",
        r"Metabolism and nutrition disorders",
        r"Musculoskeletal and connective tissue disorders",
        r"Neoplasms benign, malignant, and unspecified (incl cysts and polyps)",
        r"Nervous system disorders",
        r"Pregnancy, puerperium, and perinatal conditions",
        r"Psychiatric disorders",
        r"Renal and urinary disorders",
        r"Reproductive system and breast disorders",
        r"Respiratory, thoracic, and mediastinal disorders",
        r"Skin and subcutaneous tissue disorders",
        r"Social circumstances",
        r"Surgical and medical procedures",
        r"Vascular disorders",
        r"Product issues",
        r"Positive effects",
        r"Correct review",
        r"Incorrect review",
        r"Incompatibility"
        ]


In [17]:
df.describe()

,reviews
count,929
unique,601
top,It works
freq,3


In [ ]:
def generate_prompt(df, index):
    return f'''
    Please Determine if the review is positive. If so, then classify it as:
    Positive effects
    Correct review

    If the review is not positive, list all side effects mentioned and categorize each one into the most appropriate category from the following list:
    Blood and lymphatic system disorders
    Cardiac disorders
    Congenital familial and genetic disorders
    Ear and labyrinth disorders
    Endocrine disorders
    Eye disorders
    Gastrointestinal disorders
    General disorders and administration site conditions
    Hepatobiliary disorders
    Immune system disorders
    Infections and infestations
    Injury poisoning and procedural complications
    Investigations
    Metabolism and nutrition disorders
    Musculoskeletal and connective tissue disorders
    Neoplasms benign malignant and unspecified (incl cysts and polyps)
    Nervous system disorders
    Pregnancy puerperium and perinatal conditions
    Psychiatric disorders
    Renal and urinary disorders
    Reproductive system and breast disorders
    Respiratory thoracic and mediastinal disorders
    Skin and subcutaneous tissue disorders
    Social circumstances
    Surgical and medical procedures
    Vascular disorders
    Product issues
    Incorrect review
    Incompatibility

    Based on the review, provide your classification(s): "{df['reviews'][index][:128]}"
    '''

END_INDEX = 350

df_results = pd.DataFrame()
model_response_entries = []
prompts = []
model_r = []
model_responses = []
review_index = 251

while review_index < END_INDEX:
    prompt = generate_prompt(df, review_index)

    prompt_template = f'''You are a seeker of side effects in reviews. Your task is to find effects and correlate them according to classes 6-36

    USER: {prompt}

    ASSISTANT:
    '''

    response = lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.5, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)

    print(response)
    print(response["choices"][0]["text"])

    response_text = response["choices"][0]["text"]

    matches_ = re.findall(r'ASSISTANT:(.*)', response_text, re.DOTALL)

    review_index += 1


    model_response_entry = []
    for match in matches_:
        for category in categories_list:
            if category.lower() in match.lower():
                model_response_entry.append(category)


    prompts.append(prompt)
    model_r.append(response_text)
    model_responses.append(model_response_entry)

df_results = pd.DataFrame({
    'Prompt': prompts,
    'Model Response': model_r,
    'Identified Classifications': model_responses,
})


Llama.generate: prefix-match hit


{'id': 'cmpl-14b59ac3-1cbb-4817-bd94-55608401ae2c', 'object': 'text_completion', 'created': 1706010112, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'You are a seeker of side effects in reviews. Your task is to find effects and correlate them according to classes 6-36\n\n    USER: \n    Please Determine if the review is positive. If so, then classify it as:\n    Positive effects\n    Correct review\n\n    If the review is not positive, list all side effects mentioned and categorize each one into the most appropriate category from the following list:\n    Blood and lymphatic system disorders\n    Cardiac disorders\n    Congenital familial and genetic disorders\n    Ear and labyrinth disorders\n    Endocrine disorders\n    Eye disorders\n    Gastrointestinal disorders\n    General disorders and administration site conditions\n    Hepatobiliary disor

Llama.generate: prefix-match hit


{'id': 'cmpl-606e07b1-cb79-47ab-bdbf-fb7362b01e75', 'object': 'text_completion', 'created': 1706010132, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'You are a seeker of side effects in reviews. Your task is to find effects and correlate them according to classes 6-36\n\n    USER: \n    Please Determine if the review is positive. If so, then classify it as:\n    Positive effects\n    Correct review\n\n    If the review is not positive, list all side effects mentioned and categorize each one into the most appropriate category from the following list:\n    Blood and lymphatic system disorders\n    Cardiac disorders\n    Congenital familial and genetic disorders\n    Ear and labyrinth disorders\n    Endocrine disorders\n    Eye disorders\n    Gastrointestinal disorders\n    General disorders and administration site conditions\n    Hepatobiliary disor

Llama.generate: prefix-match hit


...

Llama.generate: prefix-match hit


{'id': 'cmpl-97fa6ffc-ec29-4106-a0e0-a4d4b026b3f7', 'object': 'text_completion', 'created': 1706010504, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'You are a seeker of side effects in reviews. Your task is to find effects and correlate them according to classes 6-36\n\n    USER: \n    Please Determine if the review is positive. If so, then classify it as:\n    Positive effects\n    Correct review\n\n    If the review is not positive, list all side effects mentioned and categorize each one into the most appropriate category from the following list:\n    Blood and lymphatic system disorders\n    Cardiac disorders\n    Congenital familial and genetic disorders\n    Ear and labyrinth disorders\n    Endocrine disorders\n    Eye disorders\n    Gastrointestinal disorders\n    General disorders and administration site conditions\n    Hepatobiliary disor

Llama.generate: prefix-match hit


{'id': 'cmpl-91c62e66-5215-4286-a809-e0d1de5b6318', 'object': 'text_completion', 'created': 1706010532, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'You are a seeker of side effects in reviews. Your task is to find effects and correlate them according to classes 6-36\n\n    USER: \n    Please Determine if the review is positive. If so, then classify it as:\n    Positive effects\n    Correct review\n\n    If the review is not positive, list all side effects mentioned and categorize each one into the most appropriate category from the following list:\n    Blood and lymphatic system disorders\n    Cardiac disorders\n    Congenital familial and genetic disorders\n    Ear and labyrinth disorders\n    Endocrine disorders\n    Eye disorders\n    Gastrointestinal disorders\n    General disorders and administration site conditions\n    Hepatobiliary disor

Llama.generate: prefix-match hit


In [32]:
df_results

,Prompt,Model Response,Identified Classifications
0,\n Please Determine if the review is positi...,You are a seeker of side effects in reviews. Y...,"[Blood and lymphatic system disorders, Cardiac..."
1,\n Please Determine if the review is positi...,You are a seeker of side effects in reviews. Y...,"[Blood and lymphatic system disorders, Cardiac..."
2,\n Please Determine if the review is positi...,You are a seeker of side effects in reviews. Y...,"[Blood and lymphatic system disorders, Cardiac..."
3,\n Please Determine if the review is positi...,You are a seeker of side effects in reviews. Y...,"[Product issues, Positive effects, Correct rev..."
4,\n Please Determine if the review is positi...,You are a seeker of side effects in reviews. Y...,"[Blood and lymphatic system disorders, Positiv..."
5,\n Please Determine if the review is positi...,You are a seeker of side effects in reviews. Y...,"[Blood and lymphatic system disorders, Positiv..."
6,\n Please Determine if the review is positi...,You are a seeker of side effects in reviews. Y...,"[Blood and lymphatic system disorders, Cardiac..."
7,\n Please Determine if the review is positi...,You are a seeker of side effects in reviews. Y...,"[Product issues, Positive effects, Correct rev..."
8,\n Please Determine if the review is positi...,You are a seeker of side effects in reviews. Y...,"[Positive effects, Correct review]"
9,\n Please Determine if the review is positi...,You are a seeker of side effects in reviews. Y...,"[Blood and lymphatic system disorders, Cardiac..."


In [33]:
df_results.to_csv('tag_amazon_350.csv', index=False)
files.download('tag_amazon_350.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>